In [1]:
import os
import numpy as np
import pandas as pd

# from unidecode import unidecode

import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# importing library for plotting
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow import keras

import tensorflow  as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model

In [56]:
import pandas as pd
import glob

path = r'D:/PhD exp/Datasets/InSDN_DatasetCSV' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, encoding='cp1252', index_col=None, header=0, low_memory=False)
    li.append(df)
    print("Read Completed for ", filename)

df = pd.concat(li, axis=0, ignore_index=True)

df = df.rename(columns={' Label': 'Label'})

Read Completed for  D:/PhD exp/Datasets/InSDN_DatasetCSV\metasploitable-2.csv
Read Completed for  D:/PhD exp/Datasets/InSDN_DatasetCSV\Normal_data.csv
Read Completed for  D:/PhD exp/Datasets/InSDN_DatasetCSV\OVS.csv


In [57]:
df.describe()

,Src Port,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,343889.000000,343889.000000,343889.000000,3.438890e+05,343889.000000,343889.000000,3.438890e+05,3.438890e+05,343889.000000,343889.000000,...,343889.000000,343889.0,3.438890e+05,3.438890e+05,3.438890e+05,3.438890e+05,3.438890e+05,3.438890e+05,3.438890e+05,3.438890e+05
mean,21982.659541,8072.495366,4.961767,6.737171e+06,6.160331,6.119012,7.310557e+02,8.335005e+03,115.689356,4.447778,...,1.441541,0.0,6.385104e+04,3.105350e+04,1.086384e+05,4.185579e+04,4.708172e+06,1.443613e+05,4.843965e+06,4.581512e+06
std,23669.464108,16275.407358,4.862608,2.183354e+07,1554.168988,105.863392,6.965289e+04,3.429719e+05,666.519811,31.285832,...,39.470056,0.0,7.852718e+05,5.035601e+05,1.220601e+06,6.672299e+05,1.616113e+07,1.653491e+06,1.641612e+07,1.604560e+07
min,0.000000,0.000000,0.000000,-1.540000e+02,0.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000,0.000000,0.000000,1.700000e+01,0.000000,2.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,443.000000,80.000000,6.000000,2.530000e+03,0.000000,2.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,43920.000000,2495.000000,6.000000,1.208600e+04,2.000000,4.000000,3.000000e+01,3.000000e+01,30.000000,0.000000,...,1.000000,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,65518.000000,65389.000000,17.000000,1.200000e+08,910748.000000,34094.000000,3.160000e+07,1.070000e+08,64239.000000,3900.000000,...,11180.000000,0.0,1.040000e+08,6.830000e+07,1.040000e+08,1.040000e+08,1.200000e+08,7.300000e+07,1.200000e+08,1.200000e+08


In [58]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.3.130-200.175.2.130-38694-4444-6,192.168.3.130,38694,200.175.2.130,4444,6,10/1/2020 5:02,269709,4,5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,U2R
1,192.168.3.130-200.175.2.130-38693-4444-6,192.168.3.130,38693,200.175.2.130,4444,6,10/1/2020 5:02,268599,2,3,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,U2R
2,192.168.3.130-200.175.2.130-3632-33747-6,200.175.2.130,33747,192.168.3.130,3632,6,10/1/2020 5:02,22194,5,5,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,U2R
3,192.168.3.130-200.175.2.130-8180-38745-6,200.175.2.130,38745,192.168.3.130,8180,6,10/1/2020 1:39,9556,4,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA
4,192.168.3.130-200.175.2.130-8180-37217-6,200.175.2.130,37217,192.168.3.130,8180,6,10/1/2020 1:39,8782,4,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BFA


In [59]:
df["Label"].value_counts()

Label
Probe         98129
DDoS          73529
Normal        68424
DoS           53616
DDoS          48413
BFA            1405
Web-Attack      192
BOTNET          164
U2R              17
Name: count, dtype: int64

In [60]:
# Filter out rows with labels other than 'BENIGN' and 'DDoS'
df = df[df['Label'].isin(['Normal', 'DDoS'])]

# Display the filtered DataFrame
# Display the updated label counts
print(df['Label'].value_counts())

Label
DDoS      73529
Normal    68424
Name: count, dtype: int64


In [61]:
# Remove columns with only values of 0
useless_columns = ['Flow ID', 'Timestamp', 'Src IP', 'Dst IP', 'Src Port', 'Dst Port']
df.drop(labels=useless_columns, axis='columns', inplace=True)
print('After dropping some columns: \n\t there are {} columns and {} rows'.format(len(df.columns), len(df)))

After dropping some columns: 
	 there are 78 columns and 141953 rows


In [62]:
print(df.shape)

(141953, 78)


In [63]:
#drop na values and reset index
data_clean = df.dropna().reset_index()

# Checkng for DUPLICATE values
data_clean.drop_duplicates(keep='first', inplace = True)

data_clean['Label'].value_counts()

print("Read {} rows.".format(len(data_clean)))

Read 141953 rows.


In [64]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder


#drop na values and reset index
data_clean = df.dropna().reset_index()

# label encoding
labelencoder = LabelEncoder()
data_clean['Label'] = labelencoder.fit_transform(data_clean['Label'])

data_clean['Label'].value_counts()

data_clean.head()

,index,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,298,0,22,0,2,0.0,0.0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,299,0,55,0,2,0.0,0.0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,300,0,14,0,2,0.0,0.0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,301,0,19,0,2,0.0,0.0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,302,0,47,0,2,0.0,0.0,0,0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [65]:

data = data_clean





# Drop non-numeric columns if necessary
data = data.drop(columns=[ 'index'])  # Adjust column names as needed

# Handle infinities
data.replace([np.inf, -np.inf], np.nan, inplace=True)



# Check for any remaining NaNs
print("Remaining NaNs:", data.isna().sum().sum())

data.info()

Remaining NaNs: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141953 entries, 0 to 141952
Data columns (total 78 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Protocol           141953 non-null  int64  
 1   Flow Duration      141953 non-null  int64  
 2   Tot Fwd Pkts       141953 non-null  int64  
 3   Tot Bwd Pkts       141953 non-null  int64  
 4   TotLen Fwd Pkts    141953 non-null  float64
 5   TotLen Bwd Pkts    141953 non-null  float64
 6   Fwd Pkt Len Max    141953 non-null  int64  
 7   Fwd Pkt Len Min    141953 non-null  int64  
 8   Fwd Pkt Len Mean   141953 non-null  float64
 9   Fwd Pkt Len Std    141953 non-null  float64
 10  Bwd Pkt Len Max    141953 non-null  int64  
 11  Bwd Pkt Len Min    141953 non-null  int64  
 12  Bwd Pkt Len Mean   141953 non-null  float64
 13  Bwd Pkt Len Std    141953 non-null  float64
 14  Flow Byts/s        141953 non-null  float64
 15  Flow Pkts/s        141953 non-nul

In [66]:
data.head()

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,22,0,2,0.0,0.0,0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0,55,0,2,0.0,0.0,0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0,14,0,2,0.0,0.0,0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0,19,0,2,0.0,0.0,0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0,47,0,2,0.0,0.0,0,0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [67]:
from sklearn.ensemble import ExtraTreesClassifier
print("ETC")
dt = ExtraTreesClassifier(n_estimators=300, random_state=5, max_depth=30)

yy =  data['Label'].values
XX =data.drop(columns=['Label'])


# Training the model
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel


dt.fit(XX,yy)
# Get the feature importances
feature_importances = dt.feature_importances_

# Create a DataFrame with feature names and importances
df_feature_importances = pd.DataFrame({'Feature': XX.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance in descending order
df_feature_importances = df_feature_importances.sort_values(by='Importance', ascending=False)

# Extract the top 30 important features
selected_features = df_feature_importances.head(30)['Feature'].tolist()

ETC


In [68]:
selected_features.append('Label')
selected_features

['Protocol',
 'Down/Up Ratio',
 'Init Bwd Win Byts',
 'ACK Flag Cnt',
 'Bwd Pkts/s',
 'Bwd Pkt Len Min',
 'SYN Flag Cnt',
 'Flow Pkts/s',
 'Pkt Len Min',
 'Pkt Size Avg',
 'Bwd Pkt Len Mean',
 'Bwd Seg Size Avg',
 'Bwd Pkt Len Max',
 'Pkt Len Max',
 'Bwd Pkt Len Std',
 'Fwd Pkt Len Min',
 'TotLen Bwd Pkts',
 'Bwd PSH Flags',
 'Fwd IAT Tot',
 'Flow Duration',
 'Pkt Len Mean',
 'Pkt Len Std',
 'PSH Flag Cnt',
 'Bwd IAT Tot',
 'Bwd Header Len',
 'Fwd Act Data Pkts',
 'Fwd Pkt Len Mean',
 'Fwd IAT Max',
 'FIN Flag Cnt',
 'Subflow Bwd Pkts',
 'Label']

In [69]:
# Create a new DataFrame with the selected features
data = data[selected_features]

In [70]:
data.head()

,Protocol,Down/Up Ratio,Init Bwd Win Byts,ACK Flag Cnt,Bwd Pkts/s,Bwd Pkt Len Min,SYN Flag Cnt,Flow Pkts/s,Pkt Len Min,Pkt Size Avg,...,Pkt Len Std,PSH Flag Cnt,Bwd IAT Tot,Bwd Header Len,Fwd Act Data Pkts,Fwd Pkt Len Mean,Fwd IAT Max,FIN Flag Cnt,Subflow Bwd Pkts,Label
0,0,0,-1,0,90909.09091,0,0,90909.09091,0,0.0,...,0.0,0,22.0,0,0,0.0,0.0,0,2,0
1,0,0,-1,0,36363.63636,0,0,36363.63636,0,0.0,...,0.0,0,55.0,0,0,0.0,0.0,0,2,0
2,0,0,-1,0,142857.14290,0,0,142857.14290,0,0.0,...,0.0,0,14.0,0,0,0.0,0.0,0,2,0
3,0,0,-1,0,105263.15790,0,0,105263.15790,0,0.0,...,0.0,0,19.0,0,0,0.0,0.0,0,2,0
4,0,0,-1,0,42553.19149,0,0,42553.19149,0,0.0,...,0.0,0,47.0,0,0,0.0,0.0,0,2,0


In [71]:
data_insdn = data
# Display the shape of the random sample
print(data_insdn.shape)

(141953, 31)


In [72]:
print(data_insdn['Label'].value_counts())

Label
0    73529
1    68424
Name: count, dtype: int64


In [73]:
#save cleaned data sample to csv
data_insdn.to_csv('cleansample_insdn.csv', index=False)